In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd

In [63]:
data = pd.read_excel('../data/카테고리_분류_리뷰추가_2024-09-04.xlsx')
data.head()

,title,업종,review,검색어,category,멕시코,남미음식,양식,한식,곱창,...,인쇄업,스티커인쇄,홍차전문점,크레페,와플,미용,피부,체형관리,차,갤러리카페
0,마야,음식점,분위기 너무 좋고 맛있었던 성수맛집 마야에서 2인 스페어립 세트 에 고구마 과카몰리...,성수카페거리,멕시코>남미음식,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,봄의정원 성수점,음식점,예약하고 직장동료분들과 방문했는데 분위기가 좋았어요 추천합니다 특히 머쉬룸크림파스타...,성수카페거리,음식점>양식,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,사위식당 성수점,음식점,자극적이지 않아서 너무 맛있는 낙곱새에요재료도 듬뿍 들어있고 면사리 추가는 꼭 하세...,성수카페거리,"한식>곱창,막창,양",0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LOOOP 루프 베이커리 카페,음식점,빵도 맛있고 공간도 다양해서 좋은 카페였습니다 나만 알고 싶은 소중한 카페 평소 ...,성수카페거리,"카페,디저트>카페",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LOOOP 루프 베이커리 카페,음식점,빵도 맛있고 공간도 다양해서 좋은 카페였습니다 나만 알고 싶은 소중한 카페 평소 ...,성수카페거리,"카페,디저트>카페",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
place_list = data['title'].unique().tolist()
place_list

['마야',
 '봄의정원 성수점',
 '사위식당 성수점',
 'LOOOP 루프 베이커리 카페',
 '뚝도농원 성수본점',
 'bd버거 성수',
 '탐광',
 '을지다락 도산공원',
 '고베규카츠 서울 강남점',
 '최영희언양불고기',
 '이자카야 공감 강남역점',
 '윕 성수',
 '춘천한옥집',
 '온센 강남점',
 '일두한우정육식당',
 '이자카야 공감 가로수길점',
 '스미스가좋아하는한옥',
 '깡통만두',
 '천하보쌈',
 '갓잇 안국점',
 '도마 유즈라멘',
 '인사도담',
 '최부자보쌈',
 '고기꾼김춘배 종로직영점',
 '호호식당 익선',
 '파랑해녀집 종로본점',
 '은주정',
 '부촌육회 본점',
 '진주육회 2호점',
 '미래회관 압구정점',
 '갓잇 압구정 도산공원점',
 '풍천가 청담점',
 '코이라멘 도산점',
 '골드피쉬 딤섬퀴진',
 '오마카세오사이초밥 압구정점',
 '지노스 뉴욕 피자 압구정점',
 '압편 압구정편의점 압구정본점',
 '담택',
 '포포브레드',
 '멜라 홍대점',
 '온리원갈비 합정 직영점',
 '남영동양문 합정점',
 '삼평식당 홍대본점',
 '쿠로이야',
 'mtl 한남',
 '쪽갈비제작소',
 '꾸띠자르당',
 '김실력포차',
 '목로 이태원점',
 'bd버거 강남',
 '장인닭갈비 강남점',
 '따롱이훠궈 강남점',
 '남도애찬 강남본점',
 '일일향 강남역3호점',
 '더몰트하우스 강남역점',
 '사쿠라테이엔 마곡점',
 '호호방',
 '마곡관',
 '신세대맥주 마곡나루점',
 '이가네 양갈비 양꼬치 마곡나루점',
 '청도미나리식당 마곡나루역점',
 '도니랑김치생삼겹 마곡나루역점',
 '에머이 마곡나루점',
 '홍지로스 본점',
 '르템플',
 '우와 명동점',
 '하동관 명동본점',
 '더 스팟 패뷸러스',
 '무쇠김치삼겹 명동점',
 '두끼 명동역점',
 '명동교자 본점',
 '돼지명가 방이동점',
 '진대감 방이점',
 '깡우동 송파방이점',
 '짚불구이 본가 방이점',
 '논두렁황소곱창 방이본점'

In [65]:
review_list = data['review'].dropna().unique().tolist()

In [66]:
stop_words = pd.read_csv('../data/stopwords-ko.txt', header=None)
stop_words = stop_words[0].dropna().tolist()

In [67]:
vectorizer = TfidfVectorizer(min_df=2, stop_words=stop_words)

In [68]:
X = vectorizer.fit_transform(review_list)
feature_names = vectorizer.get_feature_names_out()
feature_names

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(


array(['10', '100', '1000', ..., '힙함이', '힙해서', '힙해요'], dtype=object)

In [69]:
review_sim = cosine_similarity(X)
review_sim[:2]

array([[1.        , 0.06889569, 0.07925482, 0.08090686, 0.07042875,
        0.04750413, 0.06304332, 0.06084069, 0.07815072, 0.05696644,
        0.04905762, 0.07873832, 0.05317142, 0.0352116 , 0.07023052,
        0.05116148, 0.06546962, 0.045103  , 0.06336127, 0.13290265,
        0.06388242, 0.04886632, 0.05458138, 0.07597676, 0.03102167,
        0.02670331, 0.04442776, 0.03807572, 0.03223445, 0.06964299,
        0.19061226, 0.04518132, 0.0482586 , 0.06075717, 0.0601002 ,
        0.03555527, 0.03198046, 0.06265667, 0.05510573, 0.08900551,
        0.04624897, 0.07792118, 0.05043797, 0.06989636, 0.01532702,
        0.04657201, 0.04874719, 0.06217101, 0.06792125, 0.04181177,
        0.03375818, 0.08063693, 0.03468772, 0.03809119, 0.09327689,
        0.06731878, 0.06460337, 0.07821957, 0.04971033, 0.03416432,
        0.08341581, 0.02511583, 0.04341489, 0.05909297, 0.07007432,
        0.07277253, 0.03794858, 0.0668864 , 0.05185505, 0.03848332,
        0.04209188, 0.06324138, 0.06461552, 0.04

In [70]:
def similar_recommend_by_review(place_id, n_recommend):
    place_index = place_id - 1
    similar_place = sorted(list(enumerate(review_sim[place_index])), key = lambda x:x[1], reverse = True)
    recommended = 1
    print(f"--- '{place_list[place_id - 1]}' 해당 지점을 검색한 분께 추천 드리는 지점 Top {n_recommend} ---\n")
    for place_info in similar_place[1:n_recommend+1]:
        place_name = place_list[place_info[0]]
        print(f'\t추천지점 {recommended}순위: {place_name}')
        recommended += 1

In [79]:
similar_recommend_by_review(150, 10)

--- '메이커스 X 이쁘다 호텔' 해당 지점을 검색한 분께 추천 드리는 지점 Top 10 ---

	추천지점 1순위: 올웨이즈 베이글
	추천지점 2순위: N285호텔 인사동
	추천지점 3순위: 파이브호텔 종로
	추천지점 4순위: 카페창덕궁
	추천지점 5순위: 만동제과 연남점
	추천지점 6순위: 멜로우
	추천지점 7순위: 호텔 더 디자이너스 홍대
	추천지점 8순위: 머스트스테이모텔 신정네거리역점
	추천지점 9순위: 서울이야기한옥
	추천지점 10순위: 윕 성수


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import codecs
import pickle
import pandas as pd

data = pd.read_excel('../data/카테고리_분류_리뷰추가_2024-09-04.xlsx')

place_list = data['title'].unique().tolist()
review_list = data['review'].dropna().unique().tolist()

stop_words = pd.read_csv('../data/stopwords-ko.txt', header=None)
stop_words = stop_words[0].dropna().tolist()

vectorizer = TfidfVectorizer(min_df=2, stop_words=stop_words)
X = vectorizer.fit_transform(review_list)
feature_names = vectorizer.get_feature_names_out()

review_sim = cosine_similarity(X)

def similar_recommend_by_review(place_id, n_recommend):
    place_index = place_id - 1
    similar_place = sorted(list(enumerate(review_sim[place_index])), key = lambda x:x[1], reverse = True)
    recommended = 1
    print(f"--- '{place_list[place_id - 1]}' 해당 지점을 검색한 분께 추천 드리는 지점 Top {n_recommend} ---\n")
    for place_info in similar_place[1:n_recommend+1]:
        place_name = place_list[place_info[0]]
        print(f'\t추천지점 {recommended}순위: {place_name}')
        recommended += 1

similar_recommend_by_review(1, 10)